In [ ]:
import pandas as pd
import numpy as np
import random as rand
from sys import float_info
from math import factorial

# Classes' definition

In [ ]:
class SingleView:
    
    def __init__(self,
                 dataset,              # single view dataset (received as a dataframe)
                 init_pheromone_value, # initial pheromone value
                 probability_value     # the relative importance of the view
                ):
        self.dataset = dataset
        self._init_pheromone_value = init_pheromone_value
        self._probability_value = probability_value

    def initialize(self):
        # number of features in the view
        self._num_features = len(self.dataset.columns)
            
        # set intensity of pheromone values on each feature
        self._pheremone = np.full(self._num_features, self._init_pheromone_value)
        
        # compute correlation values between each pair of features (Using Pearson correlation coefficient)
        self._correlation_matrix = self.dataset.corr(method='pearson')
        self._correlation_matrix = self._correlation_matrix.applymap(abs)
    
        # compute relevance values (Using term variance)
        self._relevance = self.dataset.var()

    @property
    def num_features(self):
        return self._num_features
    
    @property
    def probability_value(self):
        return self._probability_value
                 
    @probability_value.setter
    def probability_value(self, new_probability_value):
        self._probability_value = new_probability_value
                 
    def get_pheromone(self, feature_index):
        return self._pheremone[feature_index]
             
    def update_pheromone(self, feature_index, new_pheromone_value):
        self._pheremone[feature_index] = new_pheromone_value
                 
    def get_correlation(self, feature_index_1, feature_index_2):
        return self._correlation_matrix.iloc[feature_index_1, feature_index_2]
    
    def get_relevance(self, feature_index):
        return self._relevance[feature_index]
                 
    def __str__(self):
        return f"""
     Number of featurs: {self._num_features}
     Pheromone values:  {self._pheremone}
     Probability value: {self._probability_value}
"""

In [ ]:
class Agent:
    
    def __init__(self,
                 num_selected_features, # number of selected features for each agent
                ):
        self._num_selected_features = num_selected_features
    
    def reset(self):
        # selected_features is a dictionary in which
        # keys show the index of the view and 
        # values indicate the indeces of the selected featrues in a specific view
        self._selected_features = {}
        self._last_selected_feature = -1
        self._last_selected_view = -1
        self._total_relevance_value = 0
        self._total_correlation_value = 0
        self._total_performance_value = 0
    
    def initialize(self):
        self.reset()

    @property
    def selected_features(self):
        return self._selected_features

    @property
    def last_selected_feature(self):
        return self._last_selected_feature
                 
    @last_selected_feature.setter
    def last_selected_feature(self, value):
        self._last_selected_feature = value

    @property
    def last_selected_view(self):
        return self._last_selected_view
                 
    @last_selected_view.setter
    def last_selected_view(self, value):
        self._last_selected_view = value
    
    @property
    def total_relevance_value(self):
        return self._total_relevance_value
        
    @property
    def total_correlation_value(self):
        return self._total_correlation_value

    @property
    def total_performance_value(self):
        return self._total_performance_value
                         
    def add_next_feature(self, feature_index, view_index, relevance_value, correlation_value = 0):
        self._last_selected_feature = feature_index
        self._last_selected_view = view_index
        
        # add the selected feature to the selected_features array
        value = self._selected_features.get(view_index, [])
        value.append(feature_index)
        self._selected_features[view_index] = value
        
        # update current relevance and correlation values of the agent
        self._total_relevance_value += relevance_value
        self._total_correlation_value += correlation_value
    
    def evaluate_feature_subset(self, num_computed_correlation):
        self._total_relevance_value /= self._num_selected_features
        self._total_correlation_value /= num_computed_correlation
        self._total_performance_value = self._total_relevance_value / self._total_correlation_value
    
    def __str__(self):
        return f"""
     Selected features:         {self._selected_features}
     Last selected feature:     {self._last_selected_feature}     
     Last selected view:        {self._last_selected_view}
     Sum of relevance values:   {self._total_relevance_value}
     Sum of correlation values: {self._total_correlation_value}
     Total performance:         {self._total_performance_value}
"""

In [ ]:
class MultiAgentSystem:
    
    CORRELATION_ERROR = 0.0001
    RELEVANCE_ERROR = 0.0001
    
    def __init__(self,
                 num_iters,             # maximum number of iterations
                 num_agents,            # number of agents used in each view (i.e, each omics)
                 num_selected_features, # number of selected features for each agent
                 views,                 # a list of views (i.e., multi-view/multi-omics dataset)
                 alpha,                 # importance of first heuristic information (used in the state transition rule)
                 beta,                  # importance of second heuristic information (used in the state transition rule)
                 q0,                    # the constant parameter in the state transition rule 
                                        # (Selection between greedy and probability rules)
                 discount_rate          # pheromone evaporation rate
                ):
        self._num_iters = num_iters
        self._num_agents = num_agents
        self._num_selected_features = num_selected_features
        self._views = views
        self._alpha = alpha
        self._beta = beta
        self._q0 = q0
        self._discount_rate = discount_rate
        
    def initialize(self):
        # number of datasets (i.e., number of omics types)
        self._num_views = len(self._views)
        
        # total number of agents used in all views (i.e, all omics)
        self._total_num_agents = self._num_agents * self._num_views 
            
        # count the number of times that a specific feature is selected by agents in which
        # keys show the indices of the views and 
        # values indicate dictionaries that count number of times that a specific feature is selected in a specific view
        self._feature_counter = {}
        
        # count the number of correlation values between each pair of features in a feature subset
        # this value will be used in computing the agent's performance
        self._count_correlation_computation = sum(range(self._num_selected_features))
            
        # initialize agents 
        self._agents = []
        for agent_index in range(self._total_num_agents):
            agent = Agent(self._num_selected_features)
            agent.initialize()
            self._agents.append(agent)
            
        # best agent (selected based on the agents' performance)
        self._best_selected_features = {}
        self._best_performance = 0

    @property
    def num_iters(self):
        return self._num_iters
    
    @property
    def num_selected_features(self):
        return self._num_selected_features
    
    @property
    def alpha(self):
        return self._alpha
    
    @property
    def beta(self):
        return self._beta
    
    @property
    def q0(self):
        return self._q0

    @property
    def discount_rate(self):
        return self._discount_rate
        
    @property
    def num_views(self):
        return self._num_views

    @property
    def num_agents(self):
        return self._num_agents
    
    @property
    def total_num_agents(self):
        return self._total_num_agents
        
    @property
    def best_selected_features(self):
        return self._best_selected_features
    
    @best_selected_features.setter
    def best_selected_features(self, current_best_set):
        self._best_selected_features = current_best_set

    @property
    def best_performance(self):
        return self._best_performance
    
    @best_performance.setter
    def best_performance(self, current_best_performance):
        self._best_performance = current_best_performance
    
    @property
    def count_correlation_computation(self):
        return self._count_correlation_computation
    
    @property
    def views(self):
        return self._views
    
    @property
    def agents(self):
        return self._agents
    
    @property
    def feature_counter(self):
        return self._feature_counter
    
    def reset_feature_counter(self):
        self._feature_counter = {}
    
    def update_feature_counter(self, feature_index, view_index):
        view_index_value = self._feature_counter.get(view_index, {})
        view_index_value[feature_index] = view_index_value.get(feature_index, 0) + 1
        self._feature_counter[view_index] = view_index_value

    def reset_agents(self):
        for agent_index in range(self._total_num_agents):
            self._agents[agent_index].reset()
    
    def get_views_probability(self):
        views_probability = []
        for view_index in range(self._num_views):
            views_probability.append(self._views[view_index].probability_value)
        return views_probability
       
    def set_views_probability(self, views_probability):
        for view_index in range(self._num_views):
            self._views[view_index].probability_value = views_probability[view_index]
    
    def set_agents_start_nodes(self):
        total_agent_index = 0
        for view_index in range(self.num_views):
            # generate initial nodes of the agent (unique elements chosen from the list of features)
            initial_features = rand.sample(range(self.views[view_index].num_features),self.num_agents)
        
            for feature_index in initial_features:
                relevance_value = self.views[view_index].get_relevance(feature_index)
                self.agents[total_agent_index].add_next_feature(feature_index, view_index, relevance_value)
                total_agent_index += 1
                self.update_feature_counter(feature_index, view_index)
    
    def select_by_greedy_rule(self, current_agent_index):
        agent = self.agents[current_agent_index]
        current_selected_features = agent.selected_features.get(agent.last_selected_view, [])
        current_view = self.views[agent.last_selected_view]
        total_features = current_view.num_features

        max_value = -float_info.max
        max_index = -1
        
        for feature_index in range(total_features):
            if feature_index not in current_selected_features:
                correlation_value = current_view.get_correlation(agent.last_selected_feature, feature_index)
                denominator_value = pow(correlation_value + self.CORRELATION_ERROR, self.beta)
                numerator_value = pow(current_view.get_relevance(feature_index), self.alpha)
                result = (current_view.get_pheromone(feature_index) * numerator_value) / denominator_value
                
                if result > max_value:
                    max_value = result
                    max_index = feature_index
                    
        return agent.last_selected_view, max_index # A tuple (view index, feature index)
    
    def select_by_probability_rule(self, current_agent_index, views_probability):
        # select the next view based on the probability distribution
        next_view_index = rand.choices(range(self.num_views), weights=views_probability, k=1)[0]
        
        # find the candidate features in the selected view for the specific agent
        agent = self.agents[current_agent_index]
        current_selected_features = agent.selected_features.get(next_view_index, [])
        total_features_set = np.arange(self.views[next_view_index].num_features)
        candidate_features = np.setdiff1d(total_features_set, current_selected_features)
        
        current_view = self.views[agent.last_selected_view]
        next_view = self.views[next_view_index]
            
        features_probability = []
        sum_of_probabilities = 0
        for feature_index in candidate_features:
            correlation_value = np.corrcoef(current_view.dataset.iloc[:,agent.last_selected_feature],
                                            next_view.dataset.iloc[:,feature_index])[1,0]
            correlation_value = abs(correlation_value)
            denominator_value = pow(correlation_value + self.CORRELATION_ERROR, self.beta)
            numerator_value = pow(next_view.get_relevance(feature_index), self.alpha)
            result = (next_view.get_pheromone(feature_index) * numerator_value) / denominator_value
            features_probability.append(result)
            sum_of_probabilities += result
        
        features_probability = list(map(lambda x: x/sum_of_probabilities, features_probability))
        
        # select the next feature based on its probability value
        next_feature_index = rand.choices(candidate_features, weights=features_probability, k=1)[0]
        
        return next_view_index, next_feature_index # A tuple (view index, feature index)

    def apply_state_transition_rule(self, current_agent_index, views_probability):
        q = np.random.rand()
        next_view_index = -1
        next_feature_index = -1
        
        # selection between greedy and probability rules
        if (q <= self.q0):
            next_view_index, next_feature_index = self.select_by_greedy_rule(current_agent_index)
        else:
            next_view_index, next_feature_index = self.select_by_probability_rule(current_agent_index, views_probability)
          
        # compute the correlation values of the new selected feature with previous selected features by agent
        agent = self.agents[current_agent_index]
        relevance_value = self.views[next_view_index].get_relevance(next_feature_index)
        next_feature = self.views[next_view_index].dataset.iloc[:,next_feature_index]
        sum_correlation = 0
        for view_index in agent.selected_features.keys():
            for feature_index in agent.selected_features.get(view_index):
                if (view_index != next_view_index) or (feature_index != next_feature_index):
                    current_view = self.views[view_index]
                    correlation_value = np.corrcoef(current_view.dataset.iloc[:,feature_index], next_feature)[1,0]
                    sum_correlation += abs(correlation_value)
        
        return next_view_index, next_feature_index, sum_correlation, relevance_value
    
    def evaluate_candidate_subsets(self):
        for agent_index in range(self.total_num_agents):
            self.agents[agent_index].evaluate_feature_subset(self.count_correlation_computation)
    
    def update_best_selected_subset(self):
        for agent_index in range(self.total_num_agents):
            if self.best_performance <= self.agents[agent_index].total_performance_value:
                self.best_performance = self.agents[agent_index].total_performance_value
                self.best_selected_features = self.agents[agent_index].selected_features
    
    def update_pheromone_values(self):
        view_selected_feature = []
        count_selected_feature = self.total_num_agents * self.num_selected_features
        
        for view_index in range(self.num_views):
            current_view = self.views[view_index]
            view_index_value = self.feature_counter.get(view_index, {})
            best_view_index_value = self.best_selected_features.get(view_index, {})
            for feature_index in range(self.views[view_index].num_features):
                feature_index_counter = view_index_value.get(feature_index,0)
                feature_index_counter /= count_selected_feature
                
                # add additional quantity to the feature belonging to the best subset
                second_term = 0
                if feature_index in best_view_index_value:
                    second_term = self.best_performance
                
                second_term += feature_index_counter
                second_term *= self.discount_rate
                first_term = (1 - self.discount_rate) * current_view.get_pheromone(feature_index)
                new_pheromone_value = first_term + second_term

                current_view.update_pheromone(feature_index, new_pheromone_value)

    def update_views_probability_values(self, views_probability):
        view_selected_feature = []
        count_selected_feature = 0
        
        for view_index in range(self.num_views):
            view_index_value = self.feature_counter.get(view_index, {})
            numerator_value = sum(view_index_value.values())
            view_selected_feature.append(numerator_value)
            count_selected_feature += numerator_value
        
        view_selected_feature = list(map(lambda x: self.discount_rate * (x/count_selected_feature), 
                                         view_selected_feature))
        
        for view_index in range(self.num_views):
            first_term = (1 - self.discount_rate) * views_probability[view_index]
            views_probability[view_index] = first_term + view_selected_feature[view_index]
        
        sum_of_probabilities = sum(views_probability)
        views_probability = list(map(lambda x: x/sum_of_probabilities, views_probability))
        
        self.set_views_probability(views_probability)
                
    def print_agents(self):
        for agent_index in range(self.total_num_agents):
            print(f"     ########## Agent {agent_index} ###########")
            print(self._agents[agent_index])
            print("     #########################################")
    
    def print_views(self):
        for view_index in range(self.num_views):
            print(f"     ############# View {view_index} ########")
            print(self.views[view_index])
            print("     #########################################")
    
    def start(self):
        for iteration_index in range(self.num_iters):
            print(f"     ------------------------------- Iteration {iteration_index + 1} -------------------------------")
            self.reset_feature_counter()
            self.reset_agents()
            self.set_agents_start_nodes()
            views_probability = self.get_views_probability()
            
            for feature_index in range(self.num_selected_features - 1):
                print(f"                 ---------- Current selected feature {feature_index + 2} --------------------- ")
                for agent_index in range(self.total_num_agents):
                    next_view, next_feature, sum_correlation, relevance_value = self.apply_state_transition_rule(agent_index,
                                                                                                                 views_probability)
                    self.agents[agent_index].add_next_feature(next_feature, next_view, relevance_value, sum_correlation)
                    self.update_feature_counter(next_feature, next_view)
                
            self.evaluate_candidate_subsets()
            self.update_best_selected_subset()
            self.update_pheromone_values()
            self.update_views_probability_values(views_probability)
            
#             self.print_views()
#             self.print_agents()          
    
    def __str__(self):
        return f"""
     Number of iteration:         {self.num_iters}
     Number of selected features: {self.num_selected_features}     
     Alpha:                       {self.alpha}
     Beta:                        {self.beta}
     Q0:                          {self.q0}
     Discount rate:               {self.discount_rate}
     Number of views:             {self.num_views}
     Number of agents:            {self.num_agents}
     Total number of agents:      {self.total_num_agents}
     Best selected features:      {self.best_selected_features}
     Count corr computation:      {self._count_correlation_computation}
""" 

# Read Datasets

In [ ]:
#Read DNA_methylation Dataset ()
omics1 = pd.read_csv('DataSet_OvarianCancer/DNA_methylation',sep='\t',index_col=0)
omics1 = omics1.transpose()
omics1.index.names = ['sample']
omics1.columns.names = ['feature']

In [ ]:
#Read genelevel_copy_number_alteration_CNV Dataset
omics2 = pd.read_csv('DataSet_OvarianCancer/genelevel_copy_number_alteration_CNA',sep='\t',index_col=0)
omics2 = omics2.transpose()
omics2.index.names = ['sample']
omics2.columns.names = ['feature']

In [ ]:
#Read RNA-Seq Dataset
omics3 = pd.read_csv('DataSet_OvarianCancer/RNASeq',sep='\t',index_col=0)
omics3 = omics3.transpose()
omics3.index.names = ['sample']
omics3.columns.names = ['feature']

In [ ]:
def conditions(s):
    if (s['days_to_death'] / 365) >= 3:
        return 1
    elif s['vital_status'] == 'DECEASED':
        return 0
    return -1

In [ ]:
#Read clinical data
label = pd.read_csv('DataSet_OvarianCancer/ClinicalMatrix',sep='\t',index_col=0)
label = label[label['days_to_death'].notnull()]
label['survival'] = label.apply(conditions, axis=1)
label = label[label['survival'] != -1].loc[:, ['survival']]

In [ ]:
omics1.info()

In [ ]:
omics2.info()

In [ ]:
omics3.info()

In [ ]:
label.info()

In [ ]:
omics1.shape

In [ ]:
omics2.shape

In [ ]:
omics3.shape

In [ ]:
label.shape

# Keep rows (patients) that have values in all omics data

In [ ]:
common_indices = omics1.index.intersection(omics2.index)
common_indices = common_indices.intersection(omics3.index)
common_indices = common_indices.intersection(label.index)

In [ ]:
common_indices

In [ ]:
omics1_full = omics1[omics1.index.isin(common_indices)]

In [ ]:
omics1_full.info()

In [ ]:
omics2_full = omics2[omics2.index.isin(common_indices)]

In [ ]:
omics2_full.info()

In [ ]:
omics3_full = omics3[omics3.index.isin(common_indices)]

In [ ]:
omics3_full.info()

In [ ]:
label_full = label[label.index.isin(common_indices)]

In [ ]:
label_full.info()

In [ ]:
print("long-term survivors= ", sum(label_full['survival'] == 1))
print("short-term survivors= ", sum(label_full['survival'] == 0))

# Sort dataframes based on their indices

In [ ]:
omics1_final = omics1_full.sort_index(axis=0)
omics2_final = omics2_full.sort_index(axis=0)
omics3_final = omics3_full.sort_index(axis=0)
label_final = label_full.sort_index(axis=0)

In [ ]:
omics1_final.head(2)

In [ ]:
omics2_final.head(2)

In [ ]:
omics3_final.head(2)

In [ ]:
label_final.head(2)

# Pre-Filtered Step

# 1. Removed features with missing values

In [ ]:
omics1_final = omics1_final.dropna(axis=1)
omics2_final = omics2_final.dropna(axis=1)
omics3_final = omics3_final.dropna(axis=1)

In [ ]:
#DNA_methylation Dataset
print('#Remained features = ', len(omics1_final.columns))

In [ ]:
#Genelevel_copy_number_alteration_CNA Dataset
print('#Remained features = ', len(omics2_final.columns))

In [ ]:
#RNASeq Dataset
print('#Remained features = ', len(omics3_final.columns))

# 2.  Rescale feature values to lie in the interval [0,1]

In [ ]:
#Apply the min-max scaling in Pandas using the .min() and .max() methods
def min_max_scaling(df):
    df_norm = df.copy()
    for column in df_norm.columns:
        col_min_value = df_norm[column].min()
        col_max_value = df_norm[column].max()
        df_norm[column] = (df_norm[column] - col_min_value) / (col_max_value - col_min_value)
        
    return df_norm

In [ ]:
omics1_final = min_max_scaling(omics1_final)
omics2_final = min_max_scaling(omics2_final)
omics3_final = min_max_scaling(omics3_final)

# 3. Remove features with variance less than 0.05

In [ ]:
omics1_final = omics1_final.loc[:,omics1_final.var() >= 0.05]
omics2_final = omics2_final.loc[:,omics2_final.var() >= 0.05]
omics3_final = omics3_final.loc[:,omics3_final.var() >= 0.05]

In [ ]:
omics1_final.shape

In [ ]:
omics2_final.shape

In [ ]:
omics3_final.shape

# Split dataset into test and training sets
# Apply Multi-agent algorithm
# Find the best candidate feature subset
# Reduce the dataset
# Evaluate the method by different classifiers

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

In [ ]:
# get a list of models to evaluate
def get_models():
    models = {}
    models[0] = LogisticRegression()
    models[1] = RandomForestClassifier()
    return models

In [ ]:
def evaluate_model(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    predicted_results = model.predict(X_test)
    acc = accuracy_score(y_test, predicted_results)
    return acc

In [ ]:
# settings of Multi Agent algorithm
init_pheromone_value = 0.2
num_views = 3
probability_value = 1.0 / num_views

num_iters = 30
num_agents = 20
alpha = 2
beta = 2
q0 = 0.7
discount_rate = 0.2
feature_sizes = [10,20,30,40,50,60,70,80,90,100]

# repeat algorithm for different sizes of feature subsets
for feature_size in feature_sizes:
    print("\n\n***************************************************************")
    print(f"********************* Feature size {feature_size} *************************")
    print("***************************************************************")
    model_acc = {}
    # configurations to repeat the k-fold cross-validation process (designed for imbalanced Classification)
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=1)
    for train_index, test_index in cv.split(omics1_final, y=label_final):
        # split datasets into training and test sets
        omics1_final_train, omics1_final_test = omics1_final.iloc[train_index], omics1_final.iloc[test_index]
        omics2_final_train, omics2_final_test = omics2_final.iloc[train_index], omics2_final.iloc[test_index]
        omics3_final_train, omics3_final_test = omics3_final.iloc[train_index], omics3_final.iloc[test_index]
        label_final_train,  label_final_test  = label_final.iloc[train_index],  label_final.iloc[test_index]
    
        # create initial views of each dataset
        omics_view1 = SingleView(omics1_final_train, init_pheromone_value, probability_value)
        omics_view2 = SingleView(omics2_final_train, init_pheromone_value, probability_value)
        omics_view3 = SingleView(omics3_final_train, init_pheromone_value, probability_value)
        omics_view1.initialize()
        omics_view2.initialize()
        omics_view3.initialize()
        
        # start the integration feature selection algorithm using Multi Agent system
        views = [omics_view1, omics_view2, omics_view3]
        alg = MultiAgentSystem(num_iters,
                               num_agents,
                               feature_size, 
                               views,
                               alpha,
                               beta,
                               q0, 
                               discount_rate)
        alg.initialize()
        print(alg)
        alg.start()
        final_subset = alg.best_selected_features

        print(f"\n\n     Final selected subset: {final_subset}")
        
        # create reduced datasets based on final selected features
        feature_indices_view1 = final_subset.get(0, [])
        omics1_reduced_train = omics1_final_train.iloc[:,feature_indices_view1]
        omics1_reduced_test = omics1_final_test.iloc[:,feature_indices_view1]
    
        feature_indices_view2 = final_subset.get(1, [])
        omics2_reduced_train = omics2_final_train.iloc[:,feature_indices_view2]
        omics2_reduced_test = omics2_final_test.iloc[:,feature_indices_view2]
        
        feature_indices_view3 = final_subset.get(2, [])
        omics3_reduced_train = omics3_final_train.iloc[:,feature_indices_view3]
        omics3_reduced_test = omics3_final_test.iloc[:,feature_indices_view3]
    
    
        final_train_dataset = pd.concat([omics1_reduced_train, omics2_reduced_train, omics3_reduced_train], axis=1)
        final_test_dataset = pd.concat([omics1_reduced_test, omics2_reduced_test, omics3_reduced_test], axis=1)       
        
        
        # get the list of models to evaluate performance
        models = get_models()
        # evaluate each model
        new_output_file_lines = f"Feature size: {feature_size}"
        for model_index in models:
            acc = evaluate_model(models[model_index], 
                                 X_train=final_train_dataset.values,
                                 X_test=final_test_dataset.values,
                                 y_train=label_final_train.values.ravel(),
                                 y_test=label_final_test.values.ravel())
            value = model_acc.get(model_index, [])
            value.append(acc)
            model_acc[model_index] = value
            print(f"     Classifier index ({model_index}) --> classification accuracy: {acc}")
            new_output_file_lines += f"\nClassifier index ({model_index}) --> classification accuracy {model_acc[model_index]}"

        # write the current results in the output file
        output_file_lines = ""
        try:
            output_file_lines = open("output_multi_agent.txt", 'r').readlines()
            output_file_lines[-3:] = new_output_file_lines
        except Exception:
            output_file_lines = new_output_file_lines
        
        open("output_multi_agent.txt", 'w').writelines(output_file_lines)
         
            
        print("     ###########################################################################")
        print("     ###########################################################################")
    
    # prepare the output file for the next feature size
    try:
        output_file_lines = open("output_multi_agent.txt", 'r').readlines()
        output_file_lines.append("\n\n\n\n\n")
        open("output_multi_agent.txt", 'w').writelines(output_file_lines)
    except Exception:
        pass
    
    print(f"Final classification accuracies for {feature_size} selected features")
    for model_index in model_acc:
        print(f"     Classifier index ({model_index}) --> classification accuracy: {model_acc[model_index]}")